Анализ датасета и подготовка к эксперименту

In [2]:
import pandas as pd
import numpy as np

# Загрузка данных
df = pd.read_csv('/Users/maximvishnevskiy/ml_practice_7/transactions_diy.csv')
print("Размер датасета:", df.shape)
print("\nПервые 5 строк:")
print(df.head())
print("\nИнформация о столбцах:")
print(df.info())
print("\nСтатистика по числовым столбцам:")
print(df.describe())
print("\nПропущенные значения:")
print(df.isnull().sum())

Размер датасета: (1008688, 7)

Первые 5 строк:
      tr_date          bcode       client      item  \
0  01.09.2017  code000000001  client13166   sku8444   
1  01.09.2017  code000000001  client13166  sku12545   
2  01.09.2017  code000000001  client13166   sku3391   
3  01.09.2017  code000000001  client13166  sku20444   
4  01.09.2017  code000000002   client1239  sku29959   

                     item_group  quantity  amount  
0              Скобяные изделия         1      29  
1  Оборудование для сада и дачи         1     329  
2                   Инструменты         1     169  
3                   Инструменты         2     578  
4              Скобяные изделия         1     329  

Информация о столбцах:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008688 entries, 0 to 1008687
Data columns (total 7 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   tr_date     1008688 non-null  object
 1   bcode       1008688 non-null  object
 2   c

Определение целевой переменной и признаков

In [3]:
import pandas as pd
import numpy as np

# Загрузка данных
df = pd.read_csv('/Users/maximvishnevskiy/ml_practice_7/transactions_diy.csv')

# Преобразуем дату
df['tr_date'] = pd.to_datetime(df['tr_date'], dayfirst=True)
df['year_month'] = df['tr_date'].dt.to_period('M')

# Создадим целевую переменную для каждого клиента на каждый месяц:
# Был ли у клиента покупка в категории "Оборудование для сада и дачи" в следующем месяце?

# Сначала отсортируем по дате
df = df.sort_values(['client', 'tr_date'])

# Создадим признак: покупал ли в категории "Оборудование для сада и дачи" в этом месяце
df['is_garden'] = (df['item_group'] == 'Оборудование для сада и дачи').astype(int)

# Группировка по клиенту и месяцу
client_monthly = df.groupby(['client', 'year_month']).agg(
    total_amount=('amount', 'sum'),
    purchase_count=('bcode', 'nunique'),
    garden_purchase=('is_garden', 'max')
).reset_index()

# Сдвинем целевую переменную на один месяц вперёд (предсказание на следующий месяц)
client_monthly['target'] = client_monthly.groupby('client')['garden_purchase'].shift(-1)

# Удалим последний месяц для каждого клиента (там нет target)
client_monthly = client_monthly.dropna(subset=['target'])
client_monthly['target'] = client_monthly['target'].astype(int)

print("Размер client_monthly:", client_monthly.shape)
print("\nРаспределение целевой переменной:")
print(client_monthly['target'].value_counts())

Размер client_monthly: (111716, 6)

Распределение целевой переменной:
target
0    78831
1    32885
Name: count, dtype: int64


Создание дополнительных признаков и разделение данных

In [5]:
import pandas as pd
import numpy as np

# Загрузка данных
df = pd.read_csv('/Users/maximvishnevskiy/ml_practice_7/transactions_diy.csv')

# Преобразуем дату
df['tr_date'] = pd.to_datetime(df['tr_date'], dayfirst=True)
df['year_month'] = df['tr_date'].dt.to_period('M')

# Создадим целевую переменную
df['is_garden'] = (df['item_group'] == 'Оборудование для сада и дачи').astype(int)
df = df.sort_values(['client', 'tr_date'])

# Группировка по клиенту и месяцу
client_monthly = df.groupby(['client', 'year_month']).agg(
    total_amount=('amount', 'sum'),
    purchase_count=('bcode', 'nunique'),
    garden_purchase=('is_garden', 'max')
).reset_index()

# Целевая переменная: покупка в следующем месяце
client_monthly['target'] = client_monthly.groupby('client')['garden_purchase'].shift(-1)
client_monthly = client_monthly.dropna(subset=['target'])
client_monthly['target'] = client_monthly['target'].astype(int)

# Сортировка для лагов
client_monthly = client_monthly.sort_values(['client', 'year_month'])

# Лаговые признаки
for lag in [1, 2, 3]:
    client_monthly[f'garden_lag_{lag}'] = client_monthly.groupby('client')['garden_purchase'].shift(lag)
    client_monthly[f'amount_lag_{lag}'] = client_monthly.groupby('client')['total_amount'].shift(lag)
    client_monthly[f'count_lag_{lag}'] = client_monthly.groupby('client')['purchase_count'].shift(lag)

# Средние за 3 месяца
client_monthly['avg_amount_3m'] = client_monthly.groupby('client')['total_amount'].transform(lambda x: x.rolling(3, min_periods=1).mean())
client_monthly['avg_count_3m'] = client_monthly.groupby('client')['purchase_count'].transform(lambda x: x.rolling(3, min_periods=1).mean())

# Кумулятивная сумма покупок в категории
client_monthly['garden_purchase_cumsum'] = client_monthly.groupby('client')['garden_purchase'].cumsum()

# Удаляем NaN после создания лагов
client_monthly = client_monthly.dropna()

print("Размер после создания признаков:", client_monthly.shape)
print("\nКолонки:", client_monthly.columns.tolist())
print("\nПервые 3 строки:")
print(client_monthly.head(3))

Размер после создания признаков: (48945, 18)

Колонки: ['client', 'year_month', 'total_amount', 'purchase_count', 'garden_purchase', 'target', 'garden_lag_1', 'amount_lag_1', 'count_lag_1', 'garden_lag_2', 'amount_lag_2', 'count_lag_2', 'garden_lag_3', 'amount_lag_3', 'count_lag_3', 'avg_amount_3m', 'avg_count_3m', 'garden_purchase_cumsum']

Первые 3 строки:
       client year_month  total_amount  purchase_count  garden_purchase  \
7  client1000    2019-03          7896               1                0   
8  client1000    2019-07         13723               6                0   
9  client1000    2019-08          4364               3                0   

   target  garden_lag_1  amount_lag_1  count_lag_1  garden_lag_2  \
7       0           0.0        1019.0          1.0           0.0   
8       0           0.0        7896.0          1.0           0.0   
9       0           0.0       13723.0          6.0           0.0   

   amount_lag_2  count_lag_2  garden_lag_3  amount_lag_3  count_l

In [6]:
# Преобразуем год-месяц в строку
client_monthly['year_month_str'] = client_monthly['year_month'].astype(str)

# Уникальные месяцы
unique_months = sorted(client_monthly['year_month_str'].unique())
print("Уникальные месяцы:", unique_months[:5], "...", unique_months[-5:])

# Разделяем: последний месяц - тест, предпоследний - валидация, остальные - тренировка
split_month = unique_months[-1]  # самый последний месяц для теста

train_val = client_monthly[client_monthly['year_month_str'] < split_month]
test = client_monthly[client_monthly['year_month_str'] == split_month]

# Из train_val выделяем валидацию (предпоследний месяц)
val_month = unique_months[-2]
train = train_val[train_val['year_month_str'] < val_month]
val = train_val[train_val['year_month_str'] == val_month]

print("\nРазмеры:")
print("Train:", train.shape)
print("Val:", val.shape)
print("Test:", test.shape)

print("\nРаспределение target:")
print("Train:", train['target'].value_counts())
print("Val:", val['target'].value_counts())
print("Test:", test['target'].value_counts())

Уникальные месяцы: ['2017-12', '2018-01', '2018-02', '2018-03', '2018-04'] ... ['2019-05', '2019-06', '2019-07', '2019-08', '2019-09']

Размеры:
Train: (42436, 19)
Val: (3684, 19)
Test: (2825, 19)

Распределение target:
Train: target
0    28731
1    13705
Name: count, dtype: int64
Val: target
0    2722
1     962
Name: count, dtype: int64
Test: target
0    2125
1     700
Name: count, dtype: int64
